In [1]:
# Import libraries
%matplotlib inline
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns
import re
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
plt.style.use('seaborn')

In [2]:
# Import OTU data
df = pd.read_csv(open('df_all.csv','rb'),skip_blank_lines=True)

# Add 1 to all counts and log10-transform
df_plus = df.copy()
df_plus.loc[:, 'Otu8':'Otu819'] = df.loc[:, 'Otu8':'Otu819'] + 1
df_log10 = df_plus.copy()
df_log10.loc[:, 'Otu8':'Otu819'] = np.log10(df_plus.loc[:, 'Otu8':'Otu819'])

# Grab original OTU labels
OTU_columns = df_plus.loc[:, 'Otu8':'Otu819'].columns

# Import clustering results
cluster_tally = pd.read_csv('cluster_tally.csv',index_col=0)

In [3]:
# Import representative OTUs from hierarchical clustering
closest_OTUs = np.load("closest_OTUs.npy")

# Grab log10 counts of representative OTUs
rep_OTUs = df_log10[closest_OTUs]

# Append the SampleID column to the left
rep_OTUs.insert(loc=0, column='SampleID', value=df_log10.loc[:,'SampleID'].values)

# Standardize these representative OTU counts
rep_OTUs_scaled = rep_OTUs.copy()
rep_OTUs_scaled = rep_OTUs_scaled.drop(labels=['SampleID'],axis=1)

scaler = preprocessing.StandardScaler()
rep_OTUs_scaled = pd.DataFrame(scaler.fit_transform(rep_OTUs_scaled))
rep_OTUs_scaled.columns = rep_OTUs.columns[1:]

# # Sanity check after standardization:
# print("Averages are: \n{} \n".format(np.mean(rep_OTUs_scaled,axis=0))) # Sanity check
# print("Standard deviations are: \n{} \n".format(np.std(rep_OTUs_scaled,axis=0))) # Sanity check

# Finally add back the SampleID column
rep_OTUs_scaled.insert(loc=0, column='SampleID', value=df_log10.loc[:,'SampleID'].values)

In [4]:
# Import representative OTUs from Gaussian clustering
closest_OTUs_Gaussian = np.load("closest_OTUs_Gaussian.npy")

# Grab log10 counts of representative OTUs
rep_OTUs_Gaussian = df_log10[closest_OTUs_Gaussian]

# Append the SampleID column to the left
rep_OTUs_Gaussian.insert(loc=0, column='SampleID', value=df_log10.loc[:,'SampleID'].values)

# Standardize these representative OTU counts
rep_OTUs_Gaussian_scaled = rep_OTUs_Gaussian.copy()
rep_OTUs_Gaussian_scaled = rep_OTUs_Gaussian_scaled.drop(labels=['SampleID'],axis=1)

scaler = preprocessing.StandardScaler()
rep_OTUs_Gaussian_scaled = pd.DataFrame(scaler.fit_transform(rep_OTUs_Gaussian_scaled))
rep_OTUs_Gaussian_scaled.columns = rep_OTUs_Gaussian.columns[1:]

# Finally add back the SampleID column
rep_OTUs_Gaussian_scaled.insert(loc=0, column='SampleID', value=df_log10.loc[:,'SampleID'].values)

In [5]:
# Import representative OTUs from Dirichlet clustering
closest_OTUs_Dirichlet = pd.read_csv('Dirichlet_OTUs.txt',sep='\s',engine='python').columns

# Grab log10 counts of representative OTUs
rep_OTUs_Dirichlet = df_log10[closest_OTUs_Dirichlet]

# Append the SampleID column to the left
rep_OTUs_Dirichlet.insert(loc=0, column='SampleID', value=df_log10.loc[:,'SampleID'].values)

# Standardize these representative OTU counts
rep_OTUs_Dirichlet_scaled = rep_OTUs_Dirichlet.copy()
rep_OTUs_Dirichlet_scaled = rep_OTUs_Dirichlet_scaled.drop(labels=['SampleID'],axis=1)

scaler = preprocessing.StandardScaler()
rep_OTUs_Dirichlet_scaled = pd.DataFrame(scaler.fit_transform(rep_OTUs_Dirichlet_scaled))
rep_OTUs_Dirichlet_scaled.columns = rep_OTUs_Dirichlet.columns[1:]

# Finally add back the SampleID column
rep_OTUs_Dirichlet_scaled.insert(loc=0, column='SampleID', value=df_log10.loc[:,'SampleID'].values)

In [6]:
## Merge the clustered OTU data with waterchem data
# Grab waterchem data
df_waterchem = pd.read_csv(open('df_waterchem.csv','rb'),skip_blank_lines=True)

# Append OTU data to the right
df_everything_hierarch = pd.merge(df_waterchem,rep_OTUs_scaled, how='left', left_on=['SampleID'], right_on=['SampleID'])

In [7]:
df_everything_hierarch.columns

Index(['SampleID', 'EBCT', 'Ammonia_out', 'Nitrate_in', 'Nitrite_out',
       'Se_D_in', 'CODin', 'MicroC', 'Acetate', 'Reactor_1', 'Reactor_2',
       'NRR', 'NRF', 'SeRR', 'SeRF', 'Otu4353', 'Otu5669', 'Otu1447',
       'Otu4307', 'Otu81', 'Otu3754', 'Otu7065', 'Otu3705', 'Otu778', 'Otu215',
       'Otu192', 'Otu4078', 'Otu3945', 'Otu1145', 'Otu2820', 'Otu819',
       'Otu5277', 'Otu5694', 'Otu9280', 'Otu2637', 'Otu400', 'Otu5116',
       'Otu382', 'Otu4931', 'Otu195', 'Otu1682', 'Otu5862', 'Otu9070',
       'Otu1579', 'Otu86', 'Otu69', 'Otu49', 'Otu4365', 'Otu295', 'Otu9300',
       'Otu7932', 'Otu8233'],
      dtype='object')

In [8]:
# Sanity check to ensure every column is standardized
print("Averages are: \n{} \n".format(np.mean(df_everything_hierarch,axis=0))) # Sanity check
print("Standard deviations are: \n{} \n".format(np.std(df_everything_hierarch,axis=0))) # Sanity check

Averages are: 
EBCT           1.586033e-16
Ammonia_out    9.516197e-17
Nitrate_in    -3.965082e-17
Nitrite_out   -3.172066e-17
Se_D_in       -1.586033e-17
CODin         -1.586033e-17
MicroC         6.145877e-17
Acetate       -6.145877e-17
Reactor_1      5.551115e-17
Reactor_2     -5.551115e-17
NRR           -4.004733e-16
NRF            3.493237e-15
SeRR           0.000000e+00
SeRF           2.379049e-17
Otu4353       -5.749369e-17
Otu5669       -1.268826e-16
Otu1447       -2.815208e-16
Otu4307       -3.033288e-16
Otu81          8.524927e-17
Otu3754        1.268826e-16
Otu7065       -6.145877e-17
Otu3705        1.467080e-16
Otu778        -6.026925e-16
Otu215        -3.965082e-18
Otu192        -9.119689e-17
Otu4078        9.516197e-17
Otu3945       -9.516197e-17
Otu1145       -1.982541e-18
Otu2820        1.586033e-17
Otu819        -2.775558e-17
Otu5277       -3.072939e-17
Otu5694       -1.149874e-16
Otu9280       -1.784287e-17
Otu2637        3.965082e-17
Otu400         1.189525e-17
Otu51

In [9]:
# Define output variables
y = pd.DataFrame(df_everything_hierarch['SeRR'])

# Obtain categorical values of y: (y is 0 if positive, 1 if negative)
y['class'] = np.where(y['SeRR']>=0, 0, 1)
y_class = y['class'].values

# Task 1: Mean Decrease in Accuracy (MDA) experiments 

## Case 1. MDA for benchmark case (water chem variables only)

In [12]:
# Grab waterchem variables only
df_waterchem = df_everything_hierarch.loc[:,'EBCT':'Reactor_2']

# What are the waterchem variables?
waterchem_var = df_waterchem.columns

# How many waterchem variables are there?
num_waterchem_var = waterchem_var.shape[0]

print(num_waterchem_var)
print(waterchem_var)

10
Index(['EBCT', 'Ammonia_out', 'Nitrate_in', 'Nitrite_out', 'Se_D_in', 'CODin',
       'MicroC', 'Acetate', 'Reactor_1', 'Reactor_2'],
      dtype='object')


Note that each model we train corresponds to a randomly selected training/testing split, as well as a random permutation of one variable. Therefore we need to repeat this procedure $n_{trials}$ times and average over them to get a reasonable estimate of the MDA values.

### 1.1: Base-case model with no variable permutation

In [13]:
X = df_waterchem
num_waterchem_var = X.columns.shape[0]
X.columns

Index(['EBCT', 'Ammonia_out', 'Nitrate_in', 'Nitrite_out', 'Se_D_in', 'CODin',
       'MicroC', 'Acetate', 'Reactor_1', 'Reactor_2'],
      dtype='object')

In [14]:
# First, specify the number of MDA experiments to perform:
n_MDA_runs = 100

shuffled_accuracies = np.zeros([num_waterchem_var,n_MDA_runs])

In [15]:
test_frac = 0.4 # No hyperparameter selection, so no validation set
from sklearn import svm

for i in range(0,n_MDA_runs):
    ### STEP 0: Randomly split data into training and testing.

    # Split into training and testing portions
    X_train, X_test, y_train, y_test = train_test_split(X,y_class,test_size=test_frac,shuffle=True)

    acc_array = []
    
    SVM_model = svm.SVC(C=1) # Use default C value, decrease it to increase regularization, and vice versa
    SVM_model.fit(X_train,y_train)
        
    # Compute test set accuracy
    y_test_hat = SVM_model.predict(X_test)
    SVM_test_acc_waterchem = np.sum(np.equal(y_test_hat,y_test))/y_test.shape[0]*100
        
    #print("ANN testing accuracy is {:0.2f}% using SHUFFLED water chemistry variables only.".format(ANN_test_acc_waterchem*100))
    acc_array.append(SVM_test_acc_waterchem)
    acc_array = np.asarray(acc_array)
    shuffled_accuracies[:,i] = acc_array

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [16]:
# Display accuracies in all experiment in dataframe:

acc_df = pd.DataFrame(columns = ['Variable Permutated'])
acc_df['Variable Permutated'] = waterchem_var   

for i in range(0,n_MDA_runs):
    col_name = (i+1 )
    acc_df[col_name] = (shuffled_accuracies[:,i])

# Finally, calculate average accuracy values:
acc_base_df = pd.DataFrame(columns = ['Variable','Base Accuracy (%)'])
acc_base_df['Variable'] = waterchem_var

for i in acc_base_df.index:
    acc_base_df.loc[[i],'Base Accuracy (%)'] = np.average(np.asarray(acc_df.loc[[i]].values[:,1:]),axis=1)

acc_base_df['Base Accuracy (%)'] = acc_base_df.loc[:,'Base Accuracy (%)':'Base Accuracy (%)'].astype(float).round(1)
acc_base_df    

,Variable,Base Accuracy (%)
0,EBCT,91.6
1,Ammonia_out,91.6
2,Nitrate_in,91.6
3,Nitrite_out,91.6
4,Se_D_in,91.6
5,CODin,91.6
6,MicroC,91.6
7,Acetate,91.6
8,Reactor_1,91.6
9,Reactor_2,91.6


### 1.2 MDA experiment

In [17]:
df_waterchem.columns

Index(['EBCT', 'Ammonia_out', 'Nitrate_in', 'Nitrite_out', 'Se_D_in', 'CODin',
       'MicroC', 'Acetate', 'Reactor_1', 'Reactor_2'],
      dtype='object')

In [19]:
# Run MDA experiment:
X = df_waterchem
test_frac = 0.4 # No hyperparameter selection, so no validation set

for i in range(0,n_MDA_runs):
    ### STEP 0: Randomly split data into training and testing.

    # Split into training and testing portions
    X_train, X_test, y_train, y_test = train_test_split(X,y_class,test_size=test_frac,shuffle=True)

    ## Now grab the waterchem data input subset:
    X_train_water = X_train.loc[:,'EBCT':'Reactor_2']
    X_test_water = X_test.loc[:,'EBCT':'Reactor_2']

    acc_array = []
    for j in range(0,num_waterchem_var):
    ### STEP 1: PERMUTATE THE SPECIFIC VARIABLE

        # Select one of these variables to randomly permutate
        permutate_var = waterchem_var[j]

        ## For TRAINING SET:
        # Shuffle the values of this variable arbitrarily:
        values_to_shuffle = X_train_water.loc[:,permutate_var].values
        values_to_shuffle = np.random.permutation(values_to_shuffle)

        # Put these shuffled values back in the original dataset
        X_train_water_shuffled = X_train_water.copy()
        X_train_water_shuffled[permutate_var] = values_to_shuffle

        ## For TESTING SET:
        # Shuffle the values of this variable arbitrarily:
        values_to_shuffle = X_test_water.loc[:,permutate_var].values
        values_to_shuffle = np.random.permutation(values_to_shuffle)

        # Put these shuffled values back in the original dataset
        X_test_water_shuffled = X_test_water.copy()
        X_test_water_shuffled[permutate_var] = values_to_shuffle

    ### STEP 2: RE-TRAIN ANN USING SHUFFLED DATASET, THEN EVALUATE ACCURACY:

        # Train SVM:
        SVM_model = svm.SVC(C=1) # Use default C value, decrease it to increase regularization, and vice versa
        SVM_model.fit(X_train_water_shuffled,y_train)
        
        # Compute test set accuracy
        y_test_hat = SVM_model.predict(X_test_water_shuffled)
        SVM_test_acc_waterchem_shuffled = np.sum(np.equal(y_test_hat,y_test))/y_test.shape[0]*100
        
        #print("ANN testing accuracy is {:0.2f}% using SHUFFLED water chemistry variables only.".format(ANN_test_acc_waterchem*100))
        acc_array.append(SVM_test_acc_waterchem_shuffled)
    acc_array = np.asarray(acc_array)
    shuffled_accuracies[:,i] = acc_array

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

...

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [20]:
# Display accuracies in all experiment in dataframe:

acc_df = pd.DataFrame(columns = ['Variable Permutated'])
acc_df['Variable Permutated'] = waterchem_var   

for i in range(0,n_MDA_runs):
    col_name = (i+1 )
    acc_df[col_name] = (shuffled_accuracies[:,i])
    
# Finally, calculate average MDA values:
acc_avg_df = pd.DataFrame(columns = ['Variable Permutated','Avg Acc Increase (%)'])
acc_avg_df['Variable Permutated'] = waterchem_var

for i in acc_base_df.index:
    acc_avg_df.loc[[i],'Avg Acc Increase (%)'] = np.average(np.asarray(acc_df.loc[[i]].values[:,1:]),axis=1) - acc_base_df.loc[[i],'Base Accuracy (%)']

acc_avg_df['Avg Acc Increase (%)'] = acc_avg_df.loc[:,'Avg Acc Increase (%)':'Avg Acc Increase (%)'].astype(float).round(1)
acc_avg_df

,Variable Permutated,Avg Acc Increase (%)
0,EBCT,-4.8
1,Ammonia_out,-2.5
2,Nitrate_in,-0.6
3,Nitrite_out,-0.7
4,Se_D_in,-7.2
5,CODin,-0.7
6,MicroC,-1.3
7,Acetate,-1.6
8,Reactor_1,-0.0
9,Reactor_2,0.1


In [21]:
# Pick out the top waterchem and OTU variables, in terms of importance according to MDA:
num_waterchem_smallest = 4

acc_avg_df_waterchem = acc_avg_df[0:num_waterchem_var]

top_waterchem = acc_avg_df_waterchem.nsmallest(num_waterchem_smallest, 'Avg Acc Increase (%)', keep='first')

display(top_waterchem)

,Variable Permutated,Avg Acc Increase (%)
4,Se_D_in,-7.2
0,EBCT,-4.8
1,Ammonia_out,-2.5
7,Acetate,-1.6


## 2. MDA for water chem + representative OTUs from hierarchical clustering

In [22]:
# Grab waterchem variables + representative OTU variables
df_everything = df_everything_hierarch.drop(labels=['SampleID','NRR','NRF','SeRR','SeRF'],axis=1)

# What are the relevant variables?
everything_var = df_everything.columns

# How many waterchem variables are there?
num_everything_var = everything_var.shape[0]

print(num_everything_var)
print(everything_var)

47
Index(['EBCT', 'Ammonia_out', 'Nitrate_in', 'Nitrite_out', 'Se_D_in', 'CODin',
       'MicroC', 'Acetate', 'Reactor_1', 'Reactor_2', 'Otu4353', 'Otu5669',
       'Otu1447', 'Otu4307', 'Otu81', 'Otu3754', 'Otu7065', 'Otu3705',
       'Otu778', 'Otu215', 'Otu192', 'Otu4078', 'Otu3945', 'Otu1145',
       'Otu2820', 'Otu819', 'Otu5277', 'Otu5694', 'Otu9280', 'Otu2637',
       'Otu400', 'Otu5116', 'Otu382', 'Otu4931', 'Otu195', 'Otu1682',
       'Otu5862', 'Otu9070', 'Otu1579', 'Otu86', 'Otu69', 'Otu49', 'Otu4365',
       'Otu295', 'Otu9300', 'Otu7932', 'Otu8233'],
      dtype='object')


In [23]:
# First, specify the number of MDA experiments to perform:
n_MDA_runs = 100

shuffled_accuracies = np.zeros([num_everything_var,n_MDA_runs])

### 2.1: Base-case model with no variable permutation

In [24]:
X = df_everything
test_frac = 0.4 # No hyperparameter selection, so no validation set

for i in range(0,n_MDA_runs):
    ### STEP 0: Randomly split data into training and testing.

    # Split into training and testing portions
    X_train, X_test, y_train, y_test = train_test_split(X,y_class,test_size=test_frac,shuffle=True)

    acc_array = []

    # Train SVM:
    SVM_model = svm.SVC(C=1) # Use default C value, decrease it to increase regularization, and vice versa
    SVM_model.fit(X_train,y_train)
        
    # Compute test set accuracy
    y_test_hat = SVM_model.predict(X_test)
    SVM_test_acc_shuffled = np.sum(np.equal(y_test_hat,y_test))/y_test.shape[0]*100
        
    #print("ANN testing accuracy is {:0.2f}% using SHUFFLED water chemistry variables only.".format(ANN_test_acc_waterchem*100))
    acc_array.append(SVM_test_acc_shuffled)
    acc_array = np.asarray(acc_array)
    shuffled_accuracies[:,i] = acc_array

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [25]:
# Display accuracies in all experiment in dataframe:

acc_df = pd.DataFrame(columns = ['Variable Permutated'])
acc_df['Variable Permutated'] = everything_var   

for i in range(0,n_MDA_runs):
    col_name = (i+1 )
    acc_df[col_name] = (shuffled_accuracies[:,i])

# Finally, calculate average accuracy values:
acc_base_df = pd.DataFrame(columns = ['Variable','Base Accuracy (%)'])
acc_base_df['Variable'] = everything_var

for i in acc_base_df.index:
    acc_base_df.loc[[i],'Base Accuracy (%)'] = np.average(np.asarray(acc_df.loc[[i]].values[:,1:]),axis=1)

acc_base_df['Base Accuracy (%)'] = acc_base_df.loc[:,'Base Accuracy (%)':'Base Accuracy (%)'].astype(float).round(1)
acc_base_df    

,Variable,Base Accuracy (%)
0,EBCT,87.9
1,Ammonia_out,87.9
2,Nitrate_in,87.9
3,Nitrite_out,87.9
4,Se_D_in,87.9
5,CODin,87.9
6,MicroC,87.9
7,Acetate,87.9
8,Reactor_1,87.9
9,Reactor_2,87.9


### 2.2: MDA Experiment

In [26]:
# Run MDA experiment:
X = df_everything
test_frac = 0.4 # No hyperparameter selection, so no validation set

for i in range(0,n_MDA_runs):
    ### STEP 0: Randomly split data into training and testing.

    # Split into training and testing portions
    X_train, X_test, y_train, y_test = train_test_split(X,y_class,test_size=test_frac,shuffle=True)

    acc_array = []
    for j in range(0,num_everything_var):
    ### STEP 1: PERMUTATE THE SPECIFIC VARIABLE

        # Select one of these variables to randomly permutate
        permutate_var = everything_var[j]

        ## For TRAINING SET:
        # Shuffle the values of this variable arbitrarily:
        values_to_shuffle = X_train.loc[:,permutate_var].values
        values_to_shuffle = np.random.permutation(values_to_shuffle)

        # Put these shuffled values back in the original dataset
        X_train_shuffled = X_train.copy()
        X_train_shuffled[permutate_var] = values_to_shuffle

        ## For TESTING SET:
        # Shuffle the values of this variable arbitrarily:
        values_to_shuffle = X_test.loc[:,permutate_var].values
        values_to_shuffle = np.random.permutation(values_to_shuffle)

        # Put these shuffled values back in the original dataset
        X_test_shuffled = X_test.copy()
        X_test_shuffled[permutate_var] = values_to_shuffle

    ### STEP 2: RE-TRAIN ANN USING SHUFFLED DATASET, THEN EVALUATE ACCURACY:

        # Train SVM:
        SVM_model = svm.SVC(C=1) # Use default C value, decrease it to increase regularization, and vice versa
        SVM_model.fit(X_train_shuffled,y_train)
        
        # Compute test set accuracy
        y_test_hat = SVM_model.predict(X_test)
        SVM_test_acc_shuffled = np.sum(np.equal(y_test_hat,y_test))/y_test.shape[0]*100
        
        #print("ANN testing accuracy is {:0.2f}% using SHUFFLED water chemistry variables only.".format(ANN_test_acc_waterchem*100))
        acc_array.append(SVM_test_acc_shuffled)
    acc_array = np.asarray(acc_array)
    shuffled_accuracies[:,i] = acc_array

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

...

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [28]:
# Display accuracies in all experiment in dataframe:

acc_df = pd.DataFrame(columns = ['Variable Permutated'])
acc_df['Variable Permutated'] = everything_var   

for i in range(0,n_MDA_runs):
    col_name = (i+1 )
    acc_df[col_name] = (shuffled_accuracies[:,i])
    
# Finally, calculate average MDA values:
acc_avg_df = pd.DataFrame(columns = ['Variable Permutated','Avg Acc Increase (%)'])
acc_avg_df['Variable Permutated'] = everything_var

for i in acc_avg_df.index:
    acc_avg_df.loc[[i],'Avg Acc Increase (%)'] = np.average(np.asarray(acc_df.loc[[i]].values[:,1:]),axis=1) - acc_base_df.loc[[i],'Base Accuracy (%)']

acc_avg_df['Avg Acc Increase (%)'] = acc_avg_df.loc[:,'Avg Acc Increase (%)':'Avg Acc Increase (%)'].astype(float).round(1)
acc_avg_df

,Variable Permutated,Avg Acc Increase (%)
0,EBCT,-1.1
1,Ammonia_out,0.4
2,Nitrate_in,0.4
3,Nitrite_out,0.1
4,Se_D_in,-1.2
5,CODin,0.6
6,MicroC,0.4
7,Acetate,0.2
8,Reactor_1,0.3
9,Reactor_2,0.2


In [29]:
# Pick out the top waterchem and OTU variables, in terms of importance according to MDA:
num_waterchem_smallest = 4
num_OTU_smallest = 5

acc_avg_df_waterchem = acc_avg_df[0:num_waterchem_var]
acc_avg_df_OTU = acc_avg_df[num_waterchem_var:]

top_waterchem = acc_avg_df_waterchem.nsmallest(num_waterchem_smallest, 'Avg Acc Increase (%)', keep='first')
top_OTU = acc_avg_df_OTU.nsmallest(num_OTU_smallest, 'Avg Acc Increase (%)', keep='first')

display(top_waterchem)
display(top_OTU)

,Variable Permutated,Avg Acc Increase (%)
4,Se_D_in,-1.2
0,EBCT,-1.1
3,Nitrite_out,0.1
7,Acetate,0.2


,Variable Permutated,Avg Acc Increase (%)
13,Otu4307,-0.1
18,Otu778,-0.0
45,Otu7932,-0.0
11,Otu5669,0.1
14,Otu81,0.1


## 3. MDA Experiment - Water chem variables + Representative OTUs from Gaussian Mixtures

In [30]:
## Merge the clustered OTU data with waterchem data
# Grab waterchem data
df_waterchem = pd.read_csv(open('df_waterchem.csv','rb'),skip_blank_lines=True)

# Append OTU data to the right
df_everything_Gaussian = pd.merge(df_waterchem,rep_OTUs_Gaussian_scaled, how='left', left_on=['SampleID'], right_on=['SampleID'])

In [31]:
# Grab waterchem variables + representative OTU variables
df_everything = df_everything_Gaussian.drop(labels=['SampleID','NRR','NRF','SeRR','SeRF'],axis=1)

# What are the relevant variables?
everything_var = df_everything.columns

# How many waterchem variables are there?
num_everything_var = everything_var.shape[0]

print(num_everything_var)
print(everything_var)

32
Index(['EBCT', 'Ammonia_out', 'Nitrate_in', 'Nitrite_out', 'Se_D_in', 'CODin',
       'MicroC', 'Acetate', 'Reactor_1', 'Reactor_2', 'Otu3313', 'Otu31',
       'Otu49', 'Otu6', 'Otu5116', 'Otu22', 'Otu2913', 'Otu195', 'Otu4031',
       'Otu38', 'Otu3705', 'Otu1', 'Otu65', 'Otu1447', 'Otu48', 'Otu52',
       'Otu112', 'Otu7347', 'Otu23', 'Otu4273', 'Otu6945', 'Otu1145'],
      dtype='object')


### 3.1 Base-case model with no variables permutated

In [32]:
# First, specify the number of MDA experiments to perform:
n_MDA_runs = 100

shuffled_accuracies = np.zeros([num_everything_var,n_MDA_runs])

In [33]:
X = df_everything
test_frac = 0.4 # No hyperparameter selection, so no validation set

for i in range(0,n_MDA_runs):
    ### STEP 0: Randomly split data into training and testing.

    # Split into training and testing portions
    X_train, X_test, y_train, y_test = train_test_split(X,y_class,test_size=test_frac,shuffle=True)

    acc_array = []

    # Train SVM:
    SVM_model = svm.SVC(C=1) # Use default C value, decrease it to increase regularization, and vice versa
    SVM_model.fit(X_train,y_train)
        
    # Compute test set accuracy
    y_test_hat = SVM_model.predict(X_test)
    SVM_test_acc_shuffled = np.sum(np.equal(y_test_hat,y_test))/y_test.shape[0]*100
        
    #print("ANN testing accuracy is {:0.2f}% using SHUFFLED water chemistry variables only.".format(ANN_test_acc_waterchem*100))
    acc_array.append(SVM_test_acc_shuffled)
    acc_array = np.asarray(acc_array)
    shuffled_accuracies[:,i] = acc_array

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [34]:
# Display accuracies in all experiment in dataframe:

acc_df = pd.DataFrame(columns = ['Variable Permutated'])
acc_df['Variable Permutated'] = everything_var   

for i in range(0,n_MDA_runs):
    col_name = (i+1 )
    acc_df[col_name] = (shuffled_accuracies[:,i])

# Finally, calculate average accuracy values:
acc_base_df = pd.DataFrame(columns = ['Variable','Base Accuracy (%)'])
acc_base_df['Variable'] = everything_var

for i in acc_base_df.index:
    acc_base_df.loc[[i],'Base Accuracy (%)'] = np.average(np.asarray(acc_df.loc[[i]].values[:,1:]),axis=1)

acc_base_df['Base Accuracy (%)'] = acc_base_df.loc[:,'Base Accuracy (%)':'Base Accuracy (%)'].astype(float).round(1)
acc_base_df    

,Variable,Base Accuracy (%)
0,EBCT,87.7
1,Ammonia_out,87.7
2,Nitrate_in,87.7
3,Nitrite_out,87.7
4,Se_D_in,87.7
5,CODin,87.7
6,MicroC,87.7
7,Acetate,87.7
8,Reactor_1,87.7
9,Reactor_2,87.7


### 3.2 MDA Experiments

In [35]:
# First, specify the number of MDA experiments to perform:
n_MDA_runs = 100

shuffled_accuracies = np.zeros([num_everything_var,n_MDA_runs])

In [36]:
# Run MDA experiment:
X = df_everything
test_frac = 0.4 # No hyperparameter selection, so no validation set

for i in range(0,n_MDA_runs):
    ### STEP 0: Randomly split data into training and testing.

    # Split into training and testing portions
    X_train, X_test, y_train, y_test = train_test_split(X,y_class,test_size=test_frac,shuffle=True)

    acc_array = []
    for j in range(0,num_everything_var):
    ### STEP 1: PERMUTATE THE SPECIFIC VARIABLE

        # Select one of these variables to randomly permutate
        permutate_var = everything_var[j]

        ## For TRAINING SET:
        # Shuffle the values of this variable arbitrarily:
        values_to_shuffle = X_train.loc[:,permutate_var].values
        values_to_shuffle = np.random.permutation(values_to_shuffle)

        # Put these shuffled values back in the original dataset
        X_train_shuffled = X_train.copy()
        X_train_shuffled[permutate_var] = values_to_shuffle

        ## For TESTING SET:
        # Shuffle the values of this variable arbitrarily:
        values_to_shuffle = X_test.loc[:,permutate_var].values
        values_to_shuffle = np.random.permutation(values_to_shuffle)

        # Put these shuffled values back in the original dataset
        X_test_shuffled = X_test.copy()
        X_test_shuffled[permutate_var] = values_to_shuffle

    ### STEP 2: RE-TRAIN ANN USING SHUFFLED DATASET, THEN EVALUATE ACCURACY:

        # Train SVM:
        SVM_model = svm.SVC(C=1) # Use default C value, decrease it to increase regularization, and vice versa
        SVM_model.fit(X_train_shuffled,y_train)
        
        # Compute test set accuracy
        y_test_hat = SVM_model.predict(X_test_shuffled)
        SVM_test_acc_shuffled = np.sum(np.equal(y_test_hat,y_test))/y_test.shape[0]*100
        
        #print("ANN testing accuracy is {:0.2f}% using SHUFFLED water chemistry variables only.".format(ANN_test_acc_waterchem*100))
        acc_array.append(SVM_test_acc_shuffled)
    acc_array = np.asarray(acc_array)
    shuffled_accuracies[:,i] = acc_array

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

...

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [37]:
# Display accuracies in all experiment in dataframe:

acc_df = pd.DataFrame(columns = ['Variable Permutated'])
acc_df['Variable Permutated'] = everything_var   

for i in range(0,n_MDA_runs):
    col_name = (i+1 )
    acc_df[col_name] = (shuffled_accuracies[:,i])
    
# Finally, calculate average MDA values:
acc_avg_df = pd.DataFrame(columns = ['Variable Permutated','Avg Acc Increase (%)'])
acc_avg_df['Variable Permutated'] = everything_var

for i in acc_avg_df.index:
    acc_avg_df.loc[[i],'Avg Acc Increase (%)'] = np.average(np.asarray(acc_df.loc[[i]].values[:,1:]),axis=1) - acc_base_df.loc[[i],'Base Accuracy (%)']

acc_avg_df['Avg Acc Increase (%)'] = acc_avg_df.loc[:,'Avg Acc Increase (%)':'Avg Acc Increase (%)'].astype(float).round(1)
acc_avg_df

,Variable Permutated,Avg Acc Increase (%)
0,EBCT,-0.7
1,Ammonia_out,1.9
2,Nitrate_in,1.6
3,Nitrite_out,1.3
4,Se_D_in,-0.2
5,CODin,2.0
6,MicroC,1.6
7,Acetate,1.4
8,Reactor_1,1.6
9,Reactor_2,1.5


In [38]:
# Pick out the top waterchem and OTU variables, in terms of importance according to MDA:
num_waterchem_smallest = 4
num_OTU_smallest = 5

acc_avg_df_waterchem = acc_avg_df[0:num_waterchem_var]
acc_avg_df_OTU = acc_avg_df[num_waterchem_var:]

top_waterchem = acc_avg_df_waterchem.nsmallest(num_waterchem_smallest, 'Avg Acc Increase (%)', keep='first')
top_OTU = acc_avg_df_OTU.nsmallest(num_OTU_smallest, 'Avg Acc Increase (%)', keep='first')

display(top_waterchem)
display(top_OTU)

,Variable Permutated,Avg Acc Increase (%)
0,EBCT,-0.7
4,Se_D_in,-0.2
3,Nitrite_out,1.3
7,Acetate,1.4


,Variable Permutated,Avg Acc Increase (%)
15,Otu22,0.7
18,Otu4031,1.0
20,Otu3705,1.1
30,Otu6945,1.2
24,Otu48,1.3


## 4. MDA for water chem + representative OTUs from Dirichlet clusters

In [39]:
## Merge the clustered OTU data with waterchem data
# Grab waterchem data
df_waterchem = pd.read_csv(open('df_waterchem.csv','rb'),skip_blank_lines=True)

# Append OTU data to the right
df_everything_Dirichlet = pd.merge(df_waterchem,rep_OTUs_Dirichlet_scaled, how='left', left_on=['SampleID'], right_on=['SampleID'])

# Sanity check to ensure every column is standardized
print("Averages are: \n{} \n".format(np.mean(df_everything_Dirichlet,axis=0))) # Sanity check
print("Standard deviations are: \n{} \n".format(np.std(df_everything_Dirichlet,axis=0))) # Sanity check

Averages are: 
EBCT           1.586033e-16
Ammonia_out    9.516197e-17
Nitrate_in    -3.965082e-17
Nitrite_out   -3.172066e-17
Se_D_in       -1.586033e-17
CODin         -1.586033e-17
MicroC         6.145877e-17
Acetate       -6.145877e-17
Reactor_1      5.551115e-17
Reactor_2     -5.551115e-17
NRR           -4.004733e-16
NRF            3.493237e-15
SeRR           0.000000e+00
SeRF           2.379049e-17
Otu6           1.566207e-16
Otu4          -3.053113e-16
Otu9          -3.568574e-17
Otu8           4.758099e-17
Otu7           7.474180e-16
Otu2           7.077672e-16
Otu1          -9.714451e-17
Otu11          5.947623e-17
Otu12         -3.111350e-16
Otu18         -1.744636e-16
Otu10         -3.221629e-17
Otu23         -2.815208e-16
Otu13         -1.744636e-16
Otu14         -1.367953e-16
Otu8968        1.546382e-16
Otu22          3.965082e-17
Otu141        -7.137148e-17
Otu19         -1.546382e-16
Otu15         -7.137148e-17
Otu35         -2.379049e-17
dtype: float64 

Standard deviati

In [40]:
# Define output variables
y = pd.DataFrame(df_everything_Dirichlet['SeRR'])

# Obtain categorical values of y: (y is 0 if positive, 1 if negative)
y['class'] = np.where(y['SeRR']>=0, 0, 1)
y_class = y['class'].values

# Grab waterchem variables + representative OTU variables
df_everything = df_everything_Dirichlet.drop(labels=['SampleID','NRR','NRF','SeRR','SeRF'],axis=1)

# What are the relevant variables?
everything_var = df_everything.columns

# How many waterchem variables are there?
num_everything_var = everything_var.shape[0]

print(num_everything_var)
print(everything_var)

30
Index(['EBCT', 'Ammonia_out', 'Nitrate_in', 'Nitrite_out', 'Se_D_in', 'CODin',
       'MicroC', 'Acetate', 'Reactor_1', 'Reactor_2', 'Otu6', 'Otu4', 'Otu9',
       'Otu8', 'Otu7', 'Otu2', 'Otu1', 'Otu11', 'Otu12', 'Otu18', 'Otu10',
       'Otu23', 'Otu13', 'Otu14', 'Otu8968', 'Otu22', 'Otu141', 'Otu19',
       'Otu15', 'Otu35'],
      dtype='object')


### 4.1 Base-case model with no variables permutated

In [41]:
# First, specify the number of MDA experiments to perform:
n_MDA_runs = 100

shuffled_accuracies = np.zeros([num_everything_var,n_MDA_runs])

In [42]:
X = df_everything
test_frac = 0.4 # No hyperparameter selection, so no validation set

for i in range(0,n_MDA_runs):
    ### STEP 0: Randomly split data into training and testing.

    # Split into training and testing portions
    X_train, X_test, y_train, y_test = train_test_split(X,y_class,test_size=test_frac,shuffle=True)

    acc_array = []

    # Train SVM:
    SVM_model = svm.SVC(C=1) # Use default C value, decrease it to increase regularization, and vice versa
    SVM_model.fit(X_train,y_train)
        
    # Compute test set accuracy
    y_test_hat = SVM_model.predict(X_test)
    SVM_test_acc_shuffled = np.sum(np.equal(y_test_hat,y_test))/y_test.shape[0]*100
        
    #print("ANN testing accuracy is {:0.2f}% using SHUFFLED water chemistry variables only.".format(ANN_test_acc_waterchem*100))
    acc_array.append(SVM_test_acc_shuffled)
    acc_array = np.asarray(acc_array)
    shuffled_accuracies[:,i] = acc_array

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [43]:
# Display accuracies in all experiment in dataframe:

acc_df = pd.DataFrame(columns = ['Variable Permutated'])
acc_df['Variable Permutated'] = everything_var   

for i in range(0,n_MDA_runs):
    col_name = (i+1 )
    acc_df[col_name] = (shuffled_accuracies[:,i])

# Finally, calculate average accuracy values:
acc_base_df = pd.DataFrame(columns = ['Variable','Base Accuracy (%)'])
acc_base_df['Variable'] = everything_var

for i in acc_base_df.index:
    acc_base_df.loc[[i],'Base Accuracy (%)'] = np.average(np.asarray(acc_df.loc[[i]].values[:,1:]),axis=1)

acc_base_df['Base Accuracy (%)'] = acc_base_df.loc[:,'Base Accuracy (%)':'Base Accuracy (%)'].astype(float).round(1)
acc_base_df    

,Variable,Base Accuracy (%)
0,EBCT,89.8
1,Ammonia_out,89.8
2,Nitrate_in,89.8
3,Nitrite_out,89.8
4,Se_D_in,89.8
5,CODin,89.8
6,MicroC,89.8
7,Acetate,89.8
8,Reactor_1,89.8
9,Reactor_2,89.8


### 4.2 MDA Experiments

In [44]:
# First, specify the number of MDA experiments to perform:
n_MDA_runs = 100

shuffled_accuracies = np.zeros([num_everything_var,n_MDA_runs])

In [45]:
# Run MDA experiment:
X = df_everything
test_frac = 0.4 # No hyperparameter selection, so no validation set

for i in range(0,n_MDA_runs):
    ### STEP 0: Randomly split data into training and testing.

    # Split into training and testing portions
    X_train, X_test, y_train, y_test = train_test_split(X,y_class,test_size=test_frac,shuffle=True)

    acc_array = []
    for j in range(0,num_everything_var):
    ### STEP 1: PERMUTATE THE SPECIFIC VARIABLE

        # Select one of these variables to randomly permutate
        permutate_var = everything_var[j]

        ## For TRAINING SET:
        # Shuffle the values of this variable arbitrarily:
        values_to_shuffle = X_train.loc[:,permutate_var].values
        values_to_shuffle = np.random.permutation(values_to_shuffle)

        # Put these shuffled values back in the original dataset
        X_train_shuffled = X_train.copy()
        X_train_shuffled[permutate_var] = values_to_shuffle

        ## For TESTING SET:
        # Shuffle the values of this variable arbitrarily:
        values_to_shuffle = X_test.loc[:,permutate_var].values
        values_to_shuffle = np.random.permutation(values_to_shuffle)

        # Put these shuffled values back in the original dataset
        X_test_shuffled = X_test.copy()
        X_test_shuffled[permutate_var] = values_to_shuffle

    ### STEP 2: RE-TRAIN ANN USING SHUFFLED DATASET, THEN EVALUATE ACCURACY:

        # Train SVM:
        SVM_model = svm.SVC(C=1)
        SVM_model.fit(X_train_shuffled,y_train)
        
        # Compute test set accuracy
        y_test_hat = SVM_model.predict(X_test_shuffled)
        SVM_test_acc_shuffled = np.sum(np.equal(y_test_hat,y_test))/y_test.shape[0]*100
        
        #print("ANN testing accuracy is {:0.2f}% using SHUFFLED water chemistry variables only.".format(ANN_test_acc_waterchem*100))
        acc_array.append(SVM_test_acc_shuffled)
    acc_array = np.asarray(acc_array)
    shuffled_accuracies[:,i] = acc_array

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

...

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default 

In [46]:
# Display accuracies in all experiment in dataframe:

acc_df = pd.DataFrame(columns = ['Variable Permutated'])
acc_df['Variable Permutated'] = everything_var   

for i in range(0,n_MDA_runs):
    col_name = (i+1 )
    acc_df[col_name] = (shuffled_accuracies[:,i])
    
# Finally, calculate average MDA values:
acc_avg_df = pd.DataFrame(columns = ['Variable Permutated','Avg Acc Increase (%)'])
acc_avg_df['Variable Permutated'] = everything_var

for i in acc_avg_df.index:
    acc_avg_df.loc[[i],'Avg Acc Increase (%)'] = np.average(np.asarray(acc_df.loc[[i]].values[:,1:]),axis=1) - acc_base_df.loc[[i],'Base Accuracy (%)']

acc_avg_df['Avg Acc Increase (%)'] = acc_avg_df.loc[:,'Avg Acc Increase (%)':'Avg Acc Increase (%)'].astype(float).round(1)
acc_avg_df

,Variable Permutated,Avg Acc Increase (%)
0,EBCT,-3.2
1,Ammonia_out,-0.1
2,Nitrate_in,0.1
3,Nitrite_out,-1.3
4,Se_D_in,-1.9
5,CODin,-0.1
6,MicroC,-0.2
7,Acetate,-0.3
8,Reactor_1,0.1
9,Reactor_2,-0.0


In [47]:
# Pick out the top waterchem and OTU variables, in terms of importance according to MDA:
num_waterchem_smallest = 4
num_OTU_smallest = 5

acc_avg_df_waterchem = acc_avg_df[0:num_waterchem_var]
acc_avg_df_OTU = acc_avg_df[num_waterchem_var:]

top_waterchem = acc_avg_df_waterchem.nsmallest(num_waterchem_smallest, 'Avg Acc Increase (%)', keep='first')
top_OTU = acc_avg_df_OTU.nsmallest(num_OTU_smallest, 'Avg Acc Increase (%)', keep='first')

display(top_waterchem)
display(top_OTU)

,Variable Permutated,Avg Acc Increase (%)
0,EBCT,-3.2
4,Se_D_in,-1.9
3,Nitrite_out,-1.3
7,Acetate,-0.3


,Variable Permutated,Avg Acc Increase (%)
14,Otu7,-0.5
20,Otu10,-0.5
24,Otu8968,-0.5
27,Otu19,-0.5
18,Otu12,-0.4
